## Partie 1
Implémenter la méthode des treillis pour le calcul des prix d’options (pour les différents types d’options). 
On testera la méthode sur différentes instances en faisant varier les paramètres (N : le nombre de périodes, r : taux de crédit, d et u). 
On essayera de considérer aussi des instances assez larges. 

In [39]:
import matplotlib.pyplot as plt
import numpy as np

In [40]:
def assetTreeValue(n, S, u, d):  
    asset = np.zeros((n+1,n+1))
    asset[0,0] = S
    for i in range(1,n+1):
        asset[i,0] = asset[i-1,0]*u
        for j in range(1,i+1):
            asset[i,j] = asset[i-1,j-1]*d
    return asset    

In [41]:
def calculationOptionPrice(n, assetValue, K, r, u, d, PutCall, optiontype):
    value = np.zeros((n+1,n+1))
    proba = (1+r-d)/(u-d)
    
    for j in range(n+1):
        if PutCall=="C": # Call
            payoff = max(0, (assetValue[n,j]-K))
            value[n,j] = payoff
        elif PutCall=="P": #Put
            print(j)
            payoff = max(0, (K-assetValue[n,j]))
            value[n,j] = payoff
    
    if optiontype == "USA":
         for i in range(n-1,-1,-1):
            for j in range(i+1):
                if PutCall=="P":
                    value[i,j] = max(0, K-assetValue[i,j], (1/1+r)*(proba*value[i+1,j]+(1-proba)*value[i+1,j+1]))
                elif PutCall=="C":
                    value[i,j] = max(0, assetValue[i,j]-K, (1/1+r)*(proba*value[i+1,j]+(1-proba)*value[i+1,j+1]))
    
    if optiontype == "EUR":
         for i in range(n-1,-1,-1):
            for j in range(i+1):
                value[i,j] = (1/1+r)*(proba*value[i+1,j]+(1-proba)*value[i+1,j+1])
                    
    return value

In [44]:
asset = assetTreeValue(2, 100, 1.1, 0.9)
optionvalue = calculationOptionPrice(2, asset, 91, 0.05, 1.1, 0.9, "C", "EUR")

print(asset, '\n')
print(optionvalue, '\n')
print('Option value : ', optionvalue[0][0])

[[100.   0.   0.]
 [110.  90.   0.]
 [121.  99.  81.]] 

[[21.9121875  0.         0.       ]
 [25.725      6.3        0.       ]
 [30.         8.         0.       ]] 

Option value :  21.91218750000002


## Partie 2
Ecrire le modèle qui permet de minimiser la CVaR pour une probabilité donnée  (On donne le modèle général avec un nombre d’actifs n, un nombre de scénarios S, un vecteur rendement  et un rendement minimum R. 

<img src="pics\Partie 2.png" width="480" />

In [8]:
from scipy.optimize import minimize

In [9]:
# X[0] Gamma, X[1...N] Actifs risqués 
def func_minimise_ES(X):
    sum = 0
    loose_function = 0
    for i in range(S):
        for j in range (1, X.size):
            loose_function = loose_function + max(((b[j]-Y[i])*X[j]-X[0]),0)
    return X[0] + (1/(1-alpha)*S)*loose_function

In [10]:
# S fois cette contrainte
def constraint1(X,s):
    sum = 0
    for j in range (1,X.size):
        sum = sum + (b[j]-Y[s])*X[j]
        
    return max((sum-X[0]),0) - (sum-X[0])

In [11]:
# S fois cette contrainte
def constraint2(X,s):
    sum = 0
    for j in range (1,X.size):
        sum = sum + (b[j]-Y[s])*X[j]
    
    return max((sum-X[0]),0)-0

In [12]:
def constraint3(X):
    sum = 1
    for i in range(1,X.size):
        sum = sum - X[i]
    
    return sum

In [13]:
def minimise_ES_constraint4(X):
    sum = 0
    for i in range(1,X.size):
        sum += X[i]*E[i]
        
    return sum - R

#### Fonctions pour la compréhension logique : 

In [14]:
def rendement(X):
    sum = 0
    for i in range(1,X.size):
        sum += X[i]*E[i]
        
    return sum

In [15]:
def loose_function(X):
    loose_function = 0
    for i in range(S):
        for j in range (1, X.size):
            loose_function = loose_function + max(((b[j]-Y[i])*X[j]-X[0]),0)
    return loose_function

#### Générateur de scénarios :

In [16]:
import random as rd

In [17]:
def generateY(S):
    Y = []
    for i in range(S):
        Y.append(rd.normalvariate(0,3)/100) 
    return Y
S= 5
print(generateY(5))

[-0.049225342276198084, -0.002087780685745456, -0.0006247645524452924, 0.05021041805547016, 0.010321103848131145]


### Exécution

#### General Informations : 

In [18]:
alpha = 0.99 # Probabilité
R = 0.10 # Rentabilité

#### Assets Informations :

In [23]:
Nb_Assets = 5

X = np.array([1, 0.20, 0.20, 0.20, 0.20, 0.20]) # Réparitition des actifs,  Attention : X[0] == gamma
b = np.array([0, 10, 10, 10, 10, 10]) # Prix des obligations, b[0]=0 pour la fonction objective
E = np.array([0, 0.1073, 0.00415, 0.0627, 0.1045, 0.1235]) # Espérence de rendement des actifs

print('Réparition des actifs (X[0] => Gamma) X: ', X)
print('Prix des obligations b : ', b)
print('Espérance de rendement E : ', E)

Réparition des actifs (X[0] => Gamma) X:  [1.  0.2 0.2 0.2 0.2 0.2]
Prix des obligations b :  [ 0 10 10 10 10 10]
Espérance de rendement E :  [0.      0.1073  0.00415 0.0627  0.1045  0.1235 ]


#### Scénarios Informations :

In [24]:
S = 5 # Nombre de scénarios
Y = np.array(generateY(5)) # vecteur de probabilités
print('Vecteur de probabilité Y : ', Y)

Vecteur de probabilité Y :  [-0.0458266  -0.00834546 -0.04438068 -0.00360416 -0.0051126 ]


#### Lancement :

In [25]:
print('Function initale (CVaR): ', func_minimise_ES(X))
print('Function de perte initial : ', loose_function(X))
print('Gamma initial : ', X[0])
print('1 - Répartition initiale : ', constraint3(X))
print('Rendement initial : ', rendement(X))
print('Rendement initial - Rendement espéré : ', minimise_ES_constraint4(X), '\n')



# constraints
Const1 = [] # S*contrainte 
Const2 = [] # S*contrainte
for i in range(S):
    Const1.append({'type':'ineq','args' : (i,) , 'fun': constraint1})
    Const2.append({'type':'ineq','args' : (i,)  , 'fun': constraint2}) 

ineq_cons = []
ineq_cons = (Const1 + Const2 + [{'type':'eq','fun': constraint3}, {'type':'ineq','fun': minimise_ES_constraint4}])


# bonds 
bond = (0, 1.0)
ineq_bond = ((None, None),)
for i in range(Nb_Assets):
    ineq_bond = ineq_bond + (bond,) # x1 >= 0, x2 >= 0, ...


sol = minimize(func_minimise_ES,
               x0=X, 
               method='SLSQP',  # Il faut trouver la bonne méthode
               bounds=ineq_bond, 
               options={'disp': True},
               constraints=ineq_cons)
print(sol, '\n')
Res = sol.x

print('Function finale (CVaR) : ', func_minimise_ES(Res))
print('Function de perte finale : ', loose_function(Res))
print('Gamma final : ', Res[0])
print('1 - Répartition obtenue : ', constraint3(Res))
print('Rendement final : ', rendement(Res))
print('Rendement final - Rendement espéré :  ', minimise_ES_constraint4(Res))


Function initale (CVaR):  12554.634752039246
Function de perte initial :  25.107269504078516
Gamma initial :  1.0
1 - Répartition initiale :  5.551115123125783e-17
Rendement initial :  0.08043
Rendement initial - Rendement espéré :  -0.019570000000000004 

Optimization terminated successfully    (Exit mode 0)
            Current function value: 10.003604157896604
            Iterations: 3
            Function evaluations: 21
            Gradient evaluations: 3
     fun: 10.003604157896604
     jac: array([1., 0., 0., 0., 0., 0.])
 message: 'Optimization terminated successfully'
    nfev: 21
     nit: 3
    njev: 3
  status: 0
 success: True
       x: array([10.00360416,  0.25709049,  0.03907155,  0.16282352,  0.25117237,
        0.28984207]) 

Function finale (CVaR) :  10.003604157896604
Function de perte finale :  0
Gamma final :  10.003604157896604
1 - Répartition obtenue :  -2.220446049250313e-16
Rendement final :  0.09999999999999999
Rendement final - Rendement espéré :   -1.387778

## Partie 3

Considérons le modèle de Markowitz maximisant le rendement du portefeuille et assurant une CVaR  ne dépassant pas un certain seuil.  
Ecrire le modèle correspondant et le résoudre pour certaines données aléatoires. 

<img src="pics\Partie 3.png" width="494" />

In [26]:
from scipy.optimize import linprog

In [27]:
# Définition des nouvelles varibales 

J = np.array([0, 0.50, 0.25, 0.25]) # Ensemble d'indices sur les différentes valeurs disponibles
U = np.array([0, 1, 3, 4 ]) # Risque maximal toléré pour une valeur Alpha j

In [28]:
# X[0] Gamma, X[1...N] Actifs risqués 
def func_maximise_yield(X):
    rendement = 0
    for i in range(1,X.size):
        rendement += X[i]*E[i]
    return rendement

In [29]:
# S fois cette contrainte
def maximise_yield_constraint1(X,alpha_j):
    sum = 0
    for i in range(S):
        loose_function = 0
        for j in range (1, X.size):
            loose_function = loose_function + max(((b[j]-Y[i])*X[j]-X[0]),0)
            
    return U[alpha_j] - (X[0] + (1/(1-alpha)*S)*loose_function) 

In [47]:
# constraints
Max_Const1 = [] # S*contrainte 
Max_Const2 = [] # S*contrainte 
Max_Const3 = [] # S*contrainte

for i in range(1, J.size):
    Max_Const1.append({'type':'ineq','args' : (i,) , 'fun': maximise_yield_constraint1})
        
for i in range(S):
    Max_Const2.append({'type':'ineq','args' : (i,)  , 'fun': constraint2})
    Max_Const3.append({'type':'ineq','args' : (i,)  , 'fun': constraint2})

Max_ineq_cons = []
Max_ineq_cons = (Max_Const1 + Max_Const2 + Max_Const3 +[{'type':'eq','fun': constraint3}])
             


# bonds 
bond = (0, 1.0)
Max_ineq_bond = ((None, None),)
for i in range(Nb_Assets):
    Max_ineq_bond = Max_ineq_bond + (bond,) # x1 >= 0, x2 >= 0, ...


sol = minimize(func_maximise_yield,
               x0=X, 
               method='SLSQP',  # Il faut trouver la bonne méthode
               bounds=Max_ineq_bond, 
               options={'disp': True},
               constraints=Max_ineq_cons)
print(sol, '\n')

Iteration limit reached    (Exit mode 9)
            Current function value: 0.0042663511016504865
            Iterations: 100
            Function evaluations: 1280
            Gradient evaluations: 99
     fun: 0.0042663511016504865
     jac: array([0.     , 0.1073 , 0.00415, 0.0627 , 0.1045 , 0.1235 ])
 message: 'Iteration limit reached'
    nfev: 1280
     nit: 100
    njev: 99
  status: 9
 success: False
       x: array([9.98525120e+00, 2.50350475e-06, 9.98014869e-01, 1.98275174e-03,
       1.66979327e-08, 3.96356454e-09]) 

